In [1]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import VertexAIEmbeddings
from langchain.llms import VertexAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from IPython.display import display, Markdown

warnings.filterwarnings("ignore")

In [1]:
!pip3 install chromadb


  Using cached chromadb-0.3.26-py3-none-any.whl (123 kB)
  Using cached hnswlib-0.7.0.tar.gz (33 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached clickhouse_connect-0.6.3-cp39-cp39-macosx_10_9_x86_64.whl (241 kB)
  Using cached duckdb-0.8.1-cp39-cp39-macosx_10_9_x86_64.whl (14.7 MB)
  Using cached fastapi-0.98.0-py3-none-any.whl (56 kB)
  Using cached uvicorn-0.22.0-py3-none-any.whl (58 kB)
  Using cached posthog-3.0.1-py2.py3-none-any.whl (37 kB)
  Using cached pulsar_client-3.2.0-cp39-cp39-macosx_10_15_universal2.whl (10.8 MB)
  Using cached onnxruntime-1.15.1-cp39-cp39-macosx_10_15_x86_64.whl (6.8 MB)
  Using cached urllib3-2.0.3-py3-none-any.whl (123 kB)
  Using cached zstandard-0.21.0-cp39-cp39-macosx_10_9_x86_64.whl (473 kB)
  Using cached lz4-4.3.2-cp39-cp39-macosx_10_9_x86_64.whl (254 kB)
  Using cached starlette-0.27.0-py3-none-any.whl (66 kB)
  Using cached coloredlogs-15.

In [5]:
# !pip3 install p5py
# !pip3 install PEP517
!pip3 install pycairo
!apt-get install sox ffmpeg libcairo2 libcairo2-dev
!apt-get install texlive-full
!pip3 install manimlib  # or pip install manimlib

  Using cached pycairo-1.24.0.tar.gz (344 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pycairo (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-universal2-cpython-39
      creating build/lib.macosx-10.9-universal2-cpython-39/cairo
      copying cairo/__init__.py -> build/lib.macosx-10.9-universal2-cpython-39/cairo
      copying cairo/__init__.pyi -> build/lib.macosx-10.9-universal2-cpython-39/cairo
      copying cairo/py.typed -> build/lib.macosx-10.9-universal2-cpython-39/cairo
      running build_ext
      'pkg-config' not found.
      Command ['pkg-config', '--print-errors', '--exists', 'cairo >= 1.15.10']
      [end of out

In [11]:
# from langchain.document_loaders import PyPDFLoader
# from langchain.document_loaders import CSVLoader
# file = 'OutdoorClothingCatalog_1000.csv'
# loader1 = CSVLoader(file_path=file)
# pdfFile = '/OfferCreationviaGrumbles.pdf'

# loader1

In [12]:
import pathlib
desktop = pathlib.Path("./SOP")
files = []
list(desktop.iterdir())
pages1 = []
# for item in list(desktop.iterdir())[:1]:
for item in list(desktop.iterdir()):
    try:
        print(str(item))
        fileLoader = PyPDFLoader(str(item))
#         pages = fileLoader.load_and_split()
#         print(len(fileLoader.load_and_split()))
        pages1.append(fileLoader.load_and_split())
#         files.append(pages)
    except:
        print("An exception occurred")
  
print (type(pages1))


SOP/Incentivised Attach SOP.pdf
SOP/Targeted Discounts - SOP.pdf
SOP/Bump Up Offers.pdf
SOP/Casper SOP.pdf
SOP/Targeted Discount on FP - SOP.pdf
SOP/_SOP for Shipping Rule Creation.pdf
SOP/Exchange Offers.pdf
SOP/Prebook Offer.pdf
SOP/Shipping Offers - SOP.pdf
SOP/Either Or offers SOP.pdf
SOP/Promise Callout Offers - SOP.pdf
SOP/BMGN v1 Offer SOP.pdf
SOP/Callout Offer.pdf
SOP/CnC Offers - SOP.pdf
SOP/Multi Unit Listing Offers - SOP.pdf
SOP/Seller Recommendation for an Offer.pdf
SOP/Offer Creation via Grumbles .pdf
SOP/Basket Offers - SOP (1).pdf
SOP/Early Access SOP.pdf
SOP/Combo Offers - SOP.pdf
SOP/PBO SOP.pdf
<class 'list'>


In [13]:


pages = []
print(len(pages1))
for item in pages1:
    try:
        if(item.page_content):
            pages.append(item)
            print(item)
    except:
        for item2 in item:
            pages.append(item2)
print(len(pages))

# context = "\n".join(str(p.page_content) for p in pages[:7])
# print(type(context))
# print(len(context))
# # print(context)

21
207


In [14]:
vertex_llm_text = VertexAI(model_name="text-bison@001")
vertex_embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [15]:
text_splitter = CharacterTextSplitter(chunk_size=10000, chunk_overlap=0)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

In [27]:
# import Chromadb
# import chromadb.config
vector_index = Chroma.from_texts(texts, vertex_embeddings).as_retriever()

# import chromadb
# # setup Chroma in-memory, for easy prototyping. Can add persistence easily!
# client = chromadb.Client()

# # Create collection. get_collection, get_or_create_collection, delete_collection also available!
# collection = client.create_collection("all-my-documents")

# # Add docs to the collection. Can also update and delete. Row-based API coming soon!
# collection.add(
#     documents=["This is document1", "This is document2"], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
#     metadatas=[{"source": "notion"}, {"source": "google-docs"}], # filter on these!
#     ids=["doc1", "doc2"], # unique for each doc
# )

# # Query/search 2 most similar results. You can also .get by id
# results = collection.query(
#     query_texts=["This is a query document"],
#     n_results=2,
#     # where={"metadata_field": "is_equal_to_this"}, # optional filter
#     # where_document={"$contains":"search_string"}  # optional filter
# )

ValueError: Could not import chromadb python package. Please install it with `pip install chromadb`.

In [ ]:
stuff_answer = stuff_chain(
    {"input_documents": pages[:7], "question": question}, 
    return_only_outputs=True
)

In [ ]:
print(stuff_answer)

In [ ]:
# !pip install transformers

In [ ]:
final_mp_data = []

# for each document, extract metadata and intermediate steps of the MapReduce process
for doc, out in zip(
    map_reduce_outputs["input_documents"], map_reduce_outputs["intermediate_steps"]
):
    output = {}
    output["file_name"] = p(doc.metadata["source"]).stem
    output["file_type"] = p(doc.metadata["source"]).suffix
    output["page_number"] = doc.metadata["page"]
    output["chunks"] = doc.page_content
    output["answer"] = out
    final_mp_data.append(output)

In [ ]:
# create a dataframe from a dictionary
pdf_mp_answers = pd.DataFrame.from_dict(final_mp_data)
# sorting the dataframe by filename and page_number
pdf_mp_answers = pdf_mp_answers.sort_values(by=["file_name", "page_number"])
pdf_mp_answers.reset_index(inplace=True, drop=True)
pdf_mp_answers.head()
print(pdf_mp_answers)

In [ ]:
# !pip install docarray
from langchain.vectorstores import DocArrayInMemorySearch
db = DocArrayInMemorySearch.from_documents(
    pages, 
    vertex_embeddings
)

In [ ]:
# print(type(pages))

In [ ]:
### we will use above vectorstore to find answer to query
# query = "What is Incentivised offer"

In [ ]:
## find all similar document
# docs = db.similarity_search(query)
# print(len(docs))
## 4 docs are returned
# docs[0] ## indeed a shirt

In [ ]:
retriever = db.as_retriever()

In [ ]:
from vertexai.preview.language_models import TextGenerationModel


def getResponse(prompt ,temperature: float = 0.2):
    """Ideation example with a Large Language Model"""

    # TODO developer - override these parameters as needed:
    parameters = {
        "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
        "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
        "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
        "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
    }

    model = TextGenerationModel.from_pretrained("text-bison@001")
    response = model.predict(
        prompt,
        **parameters,
    )
    
    return response


In [ ]:

# from vertexai.preview.language_models import ChatModel, InputOutputTextPair

# def getChatResponse(prompt, temperature: float = 0.2):
#     chat_model = ChatModel.from_pretrained("chat-bison@001")

#     # TODO developer - override these parameters as needed:
#     parameters = {
#         "temperature": temperature,  # Temperature controls the degree of randomness in token selection.
#         "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
#         "top_p": 0.95,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
# #         "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
#     }

#     chat = chat_model.start_chat(
#         context=SOPContext,
#         examples=[
#             InputOutputTextPair(
#             ),
#         ],
#     )

#     response = chat.send_message(
#         query, **parameters
#           )
#     print(f"Response from Model: {response.text}")


In [ ]:
# query = "how does deferred Giveaway works for offer"
# query = "how does I create a Promise Callout Offer"
query = "What is insufficient participation ?"
relevantDocs = db.max_marginal_relevance_search(query, k=20)
# relevantDocs = db.similarity_search_with_score(query, k=10)

print(len(relevantDocs))
# print(relevantDocs)

qdocs = "".join([relevantDocs[i].page_content for i in range(len(relevantDocs))])
# print(qdocs)
for i in range(0,len(qdocs)-2):
    print(relevantDocs[i])
    relevantDocs[i].page_content
    print(item, "======\n")

In [ ]:

role = """
You are offer related query answering module for promotions platform on Flipkart.\
You answer questions with a high degree of precision & wherever you do not have enough precision, \
you always provide an output saying the answers might not be fully correct. \
Promotions Platform has capabilities to create and maintain an offer via its \
offer creation console- 'Grumbles' and maintenance console-- 'Casper'. Business users\
can ask multiple questions related to capabilities for creating and maintaining an offer.
"""

prompt = f""" 
Role: {role}
Follow exactly these 3 steps:
1. Read the context below and aggregrate this data
Context : {qdocs}
2. Answer the question using only this context
3. Show the source for your answers
User Question: {query}

If you don't have any context and are unsure of the answer, reply that you don't know about this topic.

"""
print(len(prompt))

In [ ]:
print(getResponse(prompt))

In [ ]:

# response = vertex_llm_text.call_as_llm(f"{qdocs} Question: What is Incentivised offer.") 

# print (sopContext)

# print(prompt)
# question =  f"{qdocs} {prompt}"
# print(question)
# print(getResponse(question))
# display(Markdown(response))

# getChatResponse()

In [ ]:
!gcloud auth application-default login